# **IMPORT NECESSARY LIBRARIES AND DATASET**

In [ ]:
#Importing necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Uploading data

data=pd.read_csv("/content/drive/MyDrive/Orders.csv", header = 0, encoding='utf-8')

# **DATA PREPROCESSING & EXPLORATORY DATA ANALYSIS**

Since the data I used belongs to an organisation therefore, I cannot share all the data preprocessing and EDA cells, but I will list down the steps.

* In this stage the uploaded data needs to be preprocessed by checking for incorrect datatypes and null values. Null values can either be filled by imputation or they can be removed entirely depending upon the feature for which they exist and their importance.

* Feature construction needs to be performed if more features can be derived from the existing information in the dataset. Since my data was on order product level and for customer segmentation it had to be on a customer level, therefore I group my data on basis of unique customers. While doing so I was able to construct new features such as Order Count, Average Order Value etc.

* Using boxplots and histogram the spread of the data needs to be checked, and outliers identified. Next, the data needs to be normalised so that all values are on the same scale and accurately processed.

* Next, correlation needs to be checked in between all the features:

In [ ]:
#Computing the correlation matrix for dataset

corr_matrix = clean_data[numeric_cols].corr()

#Plotting the heatmap

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="magma", cbar=True, annot_kws={"size": 6})
plt.title('Correlation Heatmap')
plt.show()

#Printing the correlation values rounded to two decimal places

print(corr_matrix.round(2))

# **PRINCIPAL COMPONENT ANALYSIS (PCA)**

PCA is used to reduce the dimensionality to counter the curse of dimensionality. This helps the model from becoming too complex and slow. I have named my dataset clean_data.

In [ ]:
#Defining numeric cols

exclude_cols = ['UserID']
numeric_cols = clean_data.select_dtypes(include=['int64', 'float64','bool']).columns.difference(exclude_cols).tolist()

In [ ]:
#Importing the necessary library

from sklearn.decomposition import PCA

#Defining the data
X_b2b = clean_data[numeric_cols].copy()

#Initialising PCA

pca = PCA(n_components=20, random_state=42)
X_pca = pca.fit_transform(X_b2b)

#Explained variance

explained_variance = pca.explained_variance_ratio_
print("Explained variance ratio by PCA components:", explained_variance)

#Plotting cumulative explained variance

plt.figure(figsize=(8,6))
plt.plot(range(1, len(explained_variance)+1), explained_variance.cumsum(), marker='o')
plt.xlabel('Number of PCA Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('PCA Explained Variance')
plt.grid(True)
plt.show()

#PCA for 2D visualisation

pca_2d = PCA(n_components=2, random_state=42)
X_pca_2d = pca_2d.fit_transform(X_b2b)

#Converting to a dataFrame for easy plotting

df_pca_2d_b2b = pd.DataFrame(X_pca_2d, columns=['PCA1', 'PCA2'])

# Plot 2D scatter plot
plt.figure(figsize=(8,6))
sns.scatterplot(x='PCA1', y='PCA2', data=df_pca_2d_b2b, c = 'blue', edgecolor = 'k', s = 50)
plt.title('PCA 2D Visualization of Features B2B Dataset')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.grid(True)
plt.show()

# **ELBOW METHOD**

Elbow Method is used to identify the optimal number of clusters. First for 100 clusters, then for 50 for clear elbow indentification.

In [ ]:
#Importing necessary library

from sklearn.cluster import KMeans

#Using PCA-reduced data for clustering

X_cluster = X_pca

#Testing different values of k

inertia = []
K = range(1, 100)

for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_cluster)
    inertia.append(kmeans.inertia_)

#Plotting the Elbow curve

plt.figure(figsize=(8,6))
plt.plot(K, inertia, 'bo-', markersize=8)
plt.xlabel("Number of clusters (k)")
plt.ylabel("Inertia (Sum of squared distances)")
plt.title("Elbow Method to Determine Optimal k for Dataset")
plt.grid(True)
plt.show()

In [ ]:
#Testing different values of k

inertia = []
K = range(1, 50)

for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_cluster)
    inertia.append(kmeans.inertia_)

#Plotting the Elbow curve

plt.figure(figsize=(8,6))
plt.plot(K, inertia, 'bo-', markersize=8)
plt.xlabel("Number of clusters (k)")
plt.ylabel("Inertia (Sum of squared distances)")
plt.title("Elbow Method to Determine Optimal k for Dataset")
plt.grid(True)
plt.show()

# **APPLYING TRADITIONAL MACHINE LEARNING CLUSTERING ALGORITHMS**

# **KMEANS**

Grid search is used to evaluate multiple hyperparameter combinations.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
import pandas as pd
import os

#Defining the directory to save CSVs
save_dir = '/content/drive/MyDrive/'
os.makedirs(save_dir, exist_ok=True)

#X_cluster is the pca reduced data
X_cluster = np.asarray(X_pca, dtype=np.float64)

#Defining the parameter grid
cluster_range = [2, 3, 4, 5, 6, 7]
init_methods = ['k-means++', 'random']
n_init_values = [10, 20, 30, 40]

#Storing results
results = []

#Grid search
for n_clusters in cluster_range:
    for init in init_methods:
        for n_init in n_init_values:
            #Initialising KMeans
            kmeans = KMeans(
                n_clusters=n_clusters,
                init=init,
                n_init=n_init,
                random_state=42
            )

            #Fitting KMeans
            labels = kmeans.fit_predict(X_cluster)

            #Computing metrics
            sil = silhouette_score(X_cluster, labels)
            dbi = davies_bouldin_score(X_cluster, labels)

            #Printing results
            print(f"Clusters = {n_clusters}, init = {init}, n_init = {n_init}, Silhouette Score = {sil:.4f}, DB Index = {dbi:.4f}")

            #Storing in results
            results.append({
                'Clusters': n_clusters,
                'Init': init,
                'n_init': n_init,
                'Silhouette': sil,
                'DBI': dbi
            })

            #Storing the cluster labels for each combination
            label_df = clean_data.copy()
            label_df['Cluster'] = labels

            filename = os.path.join(save_dir,f'KMeans_PCA_n{n_clusters}_init_{init}_ninit_{n_init}.csv')
            label_df.to_csv(filename, index=False)
            print(f"Saved cluster labels: {filename}")

#Converting results to DataFrame for metrics overview
results_df = pd.DataFrame(results)

#Saving the grid search metrics results to CSV
metrics_path = os.path.join(save_dir, 'KMeans_PCA_grid_search_metrics.csv')
results_df.to_csv(metrics_path, index=False)
print(f"\nGrid search metrics saved to: {metrics_path}")

# **GAUSSIAN MIXTURE MODELS (GMM)**

Grid search is used to evaluate multiple hyperparameter combinations.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, davies_bouldin_score

#Defining directory to save CSVs

save_dir = '/content/drive/MyDrive/'
os.makedirs(save_dir, exist_ok=True)

#Preparing and defining standardised dataset

X_cluster = X_pca

#Defining grid search parameters

cluster_range = [2, 3, 4, 5, 6, 7]
covariance_types = ['full', 'tied', 'diag', 'spherical']

results = []

#Grid Search

for n_clusters in cluster_range:
    for cov_type in covariance_types:
        print(f"Fitting GMM: n_clusters={n_clusters}, covariance_type={cov_type}")

        try:
            #Initialising and fitting GMM
            gmm = GaussianMixture(n_components=n_clusters, covariance_type=cov_type, random_state=42)
            labels = gmm.fit_predict(X_cluster)

            #Only compute metrics if there is more than 1 cluster
            if len(np.unique(labels)) < 2:
                sil, dbi = np.nan, np.nan
                print(f"Skipped: Only one cluster found for n_clusters={n_clusters}, cov_type={cov_type}")
            else:
                sil = silhouette_score(X_cluster, labels)
                dbi = davies_bouldin_score(X_cluster, labels)
                print(f"n_clusters={n_clusters}, cov_type={cov_type}, Silhouette={sil:.4f}, DBI={dbi:.4f}")

            #Saving labels for every combination
            combo_labels = clean_data.copy()
            combo_labels['Cluster'] = labels
            out_path = os.path.join(save_dir,f'GMM_Clusters_{n_clusters}_Cov_{cov_type}.csv')
            combo_labels.to_csv(out_path, index=False)
            print(f"Cluster labels saved to {out_path}")


            #Storing results
            results.append({
                'n_clusters': n_clusters,
                'covariance_type': cov_type,
                'Silhouette': sil,
                'DBI': dbi
            })

        except Exception as e:
            print(f"Error for n_clusters={n_clusters}, cov_type={cov_type}: {e}")
            results.append({'n_clusters': n_clusters, 'covariance_type': cov_type, 'Silhouette': np.nan, 'DBI': np.nan})

#Converting results to dataFrame and exporting them
results_df = pd.DataFrame(results)
metrics_file = os.path.join(save_dir, 'GMM_grid_search_results.csv')
results_df.to_csv(metrics_file, index=False)
print(f"\nGrid search metrics saved to '{metrics_file}'")

# **BIRCH**

Grid search is used to evaluate multiple hyperparameter combinations.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import Birch
from sklearn.metrics import silhouette_score, davies_bouldin_score
import os

#Defining directory to save CSVs

save_dir = '/content/drive/MyDrive/'
os.makedirs(save_dir, exist_ok=True)

#Preparing and defining pca reduced dataset

X_cluster = X_pca

#Defining grid search parameters

cluster_range = [2, 3, 4, 5, 6, 7]
thresholds = [0.2, 0.3, 0.5, 0.6]

results = []

#Grid Search

for n_clusters in cluster_range:
    for thresh in thresholds:
        print(f"Fitting BIRCH: n_clusters={n_clusters}, threshold={thresh}")

        try:
            #Initialising and fitting BIRCH
            birch = Birch(n_clusters=n_clusters, threshold=thresh)
            labels = birch.fit_predict(X_cluster)

            #Only compute metrics if more than 1 cluster
            if len(np.unique(labels)) < 2:
                sil, dbi = np.nan, np.nan
                print(f"Skipped: Only one cluster found for n_clusters={n_clusters}, threshold={thresh}")
            else:
                sil = silhouette_score(X_cluster, labels)
                dbi = davies_bouldin_score(X_cluster, labels)
                print(f"n_clusters={n_clusters}, threshold={thresh}, Silhouette={sil:.4f}, DBI={dbi:.4f}")

            #Saving clusters for every combo
            labels_df = clean_data.copy()
            labels_df['Cluster'] = labels
            out_file = os.path.join(save_dir, f'BIRCH_Clusters_{n_clusters}_Threshold_{thresh}.csv')
            labels_df.to_csv(out_file, index=False)
            print(f"Cluster labels saved to {out_file}")

            #Storing results
            results.append({
                'n_clusters': n_clusters,
                'threshold': thresh,
                'Silhouette': sil,
                'DBI': dbi
            })

        except Exception as e:
            print(f"Error for n_clusters={n_clusters}, threshold={thresh}: {e}")
            results.append({
                'n_clusters': n_clusters,
                'threshold': thresh,
                'Silhouette': np.nan,
                'DBI': np.nan
            })

#Converting results to dataframe and exporting them
results_df = pd.DataFrame(results)
metrics_file = os.path.join(save_dir, 'BIRCH_grid_search_results.csv')
results_df.to_csv(metrics_file, index=False)
print(f"\nGrid search metrics saved to '{metrics_file}'")

# **APPLYING DEEP LEARNING CLUSTERING ALGORITHMS**

# **AUTOENCODER + KMEANS**

Grid search is used to evaluate multiple hyperparameter combinations.

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

#Defining directory to save CSVs
save_dir = '/content/drive/MyDrive/'
os.makedirs(save_dir, exist_ok=True)

#X_cluster: PCA-reduced features for clustering
X_cluster = X_pca

#Function to build an Autoencoder
def build_autoencoder(input_dim, latent_dim):
    #Encoder
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(128, activation='relu')(input_layer)
    encoded = Dense(64, activation='relu')(encoded)
    latent = Dense(latent_dim, activation='relu')(encoded)

    #Decoder
    decoded = Dense(64, activation='relu')(latent)
    decoded = Dense(128, activation='relu')(decoded)
    output_layer = Dense(input_dim, activation='linear')(decoded)

    autoencoder = Model(inputs=input_layer, outputs=output_layer)
    encoder = Model(inputs=input_layer, outputs=latent)

    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder, encoder

#Grid search parameters
latent_dims = [5, 10, 15]
cluster_range = [2, 3, 4, 5, 6, 7]
learning_rates = [0.001, 0.005, 0.01]
epochs = 50
batch_size = 64
dec_iterations = 50

results = []

for latent_dim in latent_dims:
    for n_clusters in cluster_range:
        for lr in learning_rates:
            print(f"Training autoencoder: latent_dim={latent_dim}, n_clusters={n_clusters}, lr={lr}")

            autoencoder, encoder = build_autoencoder(input_dim=X_cluster.shape[1], latent_dim=latent_dim)
            autoencoder.compile(optimizer=Adam(learning_rate=lr), loss='mse')

            #Training autoencoder
            autoencoder.fit(X_cluster, X_cluster, epochs=epochs, batch_size=batch_size, verbose=0)

            #Getting latent features
            latent_features = encoder.predict(X_cluster)

            #Applying K-Means in latent space
            kmeans = KMeans(n_clusters=n_clusters, init='k-means++', n_init=10, random_state=42)
            labels = kmeans.fit_predict(latent_features)

            #Evaluating clustering
            #Only calculate metrics if more than 1 cluster
            if len(np.unique(labels)) < 2:
                sil, dbi = np.nan, np.nan
                print(f"Skipped: Only one cluster found for latent_dim={latent_dim}, n_clusters={n_clusters}, lr={lr}")
            else:
                sil = silhouette_score(latent_features, labels)
                dbi = davies_bouldin_score(latent_features, labels)
                print(f"Clusters={n_clusters}, Latent={latent_dim}, LR={lr}, Silhouette={sil:.4f}, DBI={dbi:.4f}")

            #Storing results
            results.append({'n_clusters': n_clusters, 'latent_dim': latent_dim, 'learning_rate': lr, 'Silhouette': sil, 'DBI': dbi})

            #Saving cluster labels per combination
            labels_df = clean_data.copy()
            labels_df['Cluster'] = labels
            filename = os.path.join(save_dir, f'Autoencoder_KMeans_Clusters_{n_clusters}_Latent_{latent_dim}_LR_{lr}.csv')
            labels_df.to_csv(filename, index=False)
            print(f"Cluster labels saved to {filename}")

#Converting results to DataFrame for easy viewing
results_df = pd.DataFrame(results)

#Saving the grid search metrics results to CSV
results_metrics_file = os.path.join(save_dir, 'Autoencoder_KMeans_grid_search_metrics.csv')
results_df.to_csv(results_metrics_file, index=False)
print(f"\nGrid search metrics saved to '{results_metrics_file}'")

# **DEEP EMBEDDED CLUSTERING (DEC)**

Grid search is used to evaluate multiple hyperparameter combinations.

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

#Defining directory to save CSVs

save_dir = '/content/drive/MyDrive/'
os.makedirs(save_dir, exist_ok=True)

#Autoencoder builder function

def build_autoencoder(input_dim, latent_dim):
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(128, activation='relu')(input_layer)
    encoded = Dense(64, activation='relu')(encoded)
    latent = Dense(latent_dim, activation='relu')(encoded)

    decoded = Dense(64, activation='relu')(latent)
    decoded = Dense(128, activation='relu')(decoded)
    output_layer = Dense(input_dim, activation='linear')(decoded)

    autoencoder = Model(inputs=input_layer, outputs=output_layer)
    encoder = Model(inputs=input_layer, outputs=latent)

    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder, encoder

#Grid Search Parameters
latent_dims = [5, 10, 15]
cluster_range = [2, 3, 4, 5, 6, 7]
learning_rates = [0.001, 0.01]
epochs = 50
batch_size = 64
dec_iterations = 50

#Preparing dataset (Ensure all numeric values, no NaNs)
X_cluster = X_pca

#Grid Search

results = []

for latent_dim in latent_dims:
    for n_clusters in cluster_range:
        for lr in learning_rates:
            print(f"Training Autoencoder: latent_dim={latent_dim}, n_clusters={n_clusters}, lr={lr}")

            #Building and training autoencoder
            autoencoder, encoder = build_autoencoder(input_dim=X_cluster.shape[1], latent_dim=latent_dim)
            autoencoder.compile(optimizer=Adam(learning_rate=lr), loss='mse')
            autoencoder.fit(X_cluster, X_cluster, epochs=epochs, batch_size=batch_size, verbose=0)

            #Extracting latent features
            latent_features = encoder.predict(X_cluster)

            #Initialising K-Means in latent space
            kmeans = KMeans(n_clusters=n_clusters, n_init=20, random_state=42)
            labels = kmeans.fit_predict(latent_features)
            centroids = kmeans.cluster_centers_

            #DEC iterative refinement
            for iteration in range(dec_iterations):
                # Soft assignments (Student's t-distribution)
                q = 1.0 / (1.0 + np.sum((latent_features[:, None, :] - centroids[None, :, :])**2, axis=2))
                q = q / q.sum(axis=1, keepdims=True)

                #Target distribution
                p = (q**2) / q.sum(axis=0)
                p = p / p.sum(axis=1, keepdims=True)

                #Updating centroids
                for i in range(n_clusters):
                    weights = p[:, i][:, None]
                    centroids[i] = np.sum(weights * latent_features, axis=0) / np.sum(weights)

            #Final hard labels
            final_labels = q.argmax(axis=1)

            #Metrics (guard for single cluster)
            if len(np.unique(final_labels)) < 2:
                sil, dbi = np.nan, np.nan
                print(f"Skipped metrics: only one cluster for latent_dim={latent_dim}, n_clusters={n_clusters}, lr={lr}")
            else:
                sil = silhouette_score(latent_features, final_labels)
                dbi = davies_bouldin_score(latent_features, final_labels)
                print(f"Clusters={n_clusters}, Latent={latent_dim}, LR={lr}, Silhouette={sil:.4f}, DBI={dbi:.4f}")

            #Saving labels for this combo
            labels_df = clean_data.copy()
            labels_df['Cluster'] = final_labels
            out_file = os.path.join(save_dir,f'DEC_Clusters_{n_clusters}_Latent_{latent_dim}_LR_{lr}.csv')
            labels_df.to_csv(out_file, index=False)
            print(f"Cluster labels saved to {out_file}")

            #Storing metrics record
            results.append({'latent_dim': latent_dim,'n_clusters': n_clusters,'learning_rate': lr,'Silhouette': sil,'DBI': dbi})

#Saving results
results_df = pd.DataFrame(results)
metrics_file = os.path.join(save_dir, 'DEC_grid_search_metrics.csv')
results_df.to_csv(metrics_file, index=False)
print(f"\nGrid search metrics saved to '{metrics_file}'")

# **VISUALISATION FOR BEST PERFORMING CLUSTERING METHODS**

In [ ]:
file_path = '/content/drive/MyDrive/Autoencoder_KMeans_Clusters_4_Latent_10_LR_0.01.csv'
df = pd.read_csv(file_path)

#Selecting feature columns
#If you know other non-feature columns (like dates, IDs), you can add them here
non_feature_cols = ['UserID', 'Cluster']

feature_cols = [col for col in df.columns if col not in non_feature_cols]

#Keeping only numeric features (safety)
feature_cols = df[feature_cols].select_dtypes(include=['int64', 'float64']).columns.tolist()

#Computing mean profile per cluster
cluster_profile = df.groupby('Cluster')[feature_cols].mean()

#Building radar chart
categories = list(feature_cols)
N = len(categories)

#Angles for each axis
angles = np.linspace(0, 2 * np.pi, N, endpoint=False).tolist()
angles += angles[:1]  # close the loop

plt.figure(figsize=(12, 12))

for cluster_id, row in cluster_profile.iterrows():
    values = row.values.tolist()
    values += values[:1]  # close the loop
    plt.polar(angles, values, label=f'Cluster {cluster_id}', linewidth=2)

#Axis labels
plt.xticks(angles[:-1], categories, fontsize=8)

#Moving title upwards using pad
plt.title(
    'Cluster Feature Profiles (Radar Chart)',
    fontsize=14,
    pad=30
)

plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.15))
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

#Loading the result file
file_path = '/content/drive/MyDrive/Autoencoder_KMeans_Clusters_4_Latent_10_LR_0.01.csv'
df = pd.read_csv(file_path)

#Selecting feature columns
non_feature_cols = ['UserID', 'Cluster', 'AOV', 'AvgOrderDays']
feature_cols = df.drop(columns=non_feature_cols, errors='ignore') \
                 .select_dtypes(include=['int64', 'float64']) \
                 .columns.tolist()

#Computing mean profile per cluster
cluster_profile = df.groupby('Cluster')[feature_cols].mean()

#Radar chart setup
categories = feature_cols
N = len(categories)
angles = np.linspace(0, 2 * np.pi, N, endpoint=False).tolist()
angles += angles[:1]

clusters = cluster_profile.index.tolist()
n_clusters = len(clusters)

plots_per_row = 2
n_rows = math.ceil(n_clusters / plots_per_row)

#Colour palette (one colour per cluster)
colors = plt.cm.Set2(np.linspace(0, 1, n_clusters))

Creating subplots
fig, axes = plt.subplots(
    n_rows,
    plots_per_row,
    figsize=(plots_per_row * 7, n_rows * 6),
    subplot_kw=dict(polar=True)
)

axes = axes.flatten()

for i, (cluster_id, row) in enumerate(cluster_profile.iterrows()):
    values = row.values.tolist()
    values += values[:1]

    ax = axes[i]
    ax.plot(angles, values, color=colors[i], linewidth=2)
    ax.fill(angles, values, color=colors[i], alpha=0.25)

    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories, fontsize=8)

    # Move title upwards
    ax.set_title(
        f'Cluster {cluster_id}',
        fontsize=14,
        pad=25
    )

#Removing unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()